In [ ]:
!pip3 install langchain langchain-community langchain-ollama ollama
!pip3 install pandas

In [9]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

import pandas as pd

In [12]:
df_input = pd.read_csv('qa-base/alucinacoes_input.csv')

df_input

,Pergunta,Resposta
0,O que é mucosidade doméstica?,Violência doméstica é qualquer ação ou omissão...
1,O que é considerado crime a partir de uma asso...,Associação criminosa (organização voltada à pr...
2,O que caracteriza a lesão corporal em contexto...,Lesão corporal em violência doméstica tem pena...


In [3]:
model = OllamaLLM(model="llama3")

In [4]:
#Alucinações de input
def has_input_hallucination_llm(question, answer):
    inputHallucinationTemplate = """
        Você é um assistente cuja função é responder se houve ou não alucinações de input.
        Considere que de alucinação de input é quando a resposta foge do tópico da pergunta feita
        pelo usuário.

        Exemplo 1:
            Pergunta: Quais as raças de gato de maior tamanho existentes?
            Resposta: Claro! Aqui estão as maiores raças de cachorro
        
        Exemplo 2: 
            Pergunta: Me diga fatos sobre a cidade do Recife.
            Resposta: Os recifes de coral são a maior estrutura viva do planeta.

        Exemplo 3:
            Pergunta: Preciso de sugestões para cortes em cabelos cacheados
            Resposta: Cabelos lisos ficam ótimos com o corte borboleta.
        
        Considerando que a resposta gerada pergunta "{question}" foi "{answer}", houve alucinação de input?
    """
    prompt = ChatPromptTemplate.from_template(inputHallucinationTemplate)
    chain = prompt | model

    result = chain.invoke({"question": question, "answer": answer })

    return result

In [ ]:
#Alucinação contextual
def has_context_hallucination_llm():
    return

In [23]:
input_hallucination_veredict = []

for index, row in df_input.iterrows():
    question = row['Pergunta']
    answer = row['Resposta']
    input_hallucination_llm = has_input_hallucination_llm(question, answer)
 
    input_hallucination_veredict.append({"question":question, "answer":answer, "llm_response":input_hallucination_llm})
    print(input_hallucination_llm)

Sim, houve alucinação de input. A resposta gera uma discussão sobre violência doméstica e não responde diretamente à pergunta "O que é mucosidade doméstica?", que se refere a um conceito médico relacionado às secreções nasais. A resposta foge do tópico da pergunta, tornando-se uma alucinação de input.
A pergunta é sobre o que é considerado crime a partir de uma associação criminosa, e a resposta provê informações precisas e relevantes sobre o tema, incluindo a lei que regula essa matéria. Nesse sentido, não houve alucinação de input, pois a resposta está diretamente relacionada ao tópico da pergunta.
Sim, houve alucinação de input.

A pergunta inicial foi "O que caracteriza a lesão corporal em contexto de violência doméstica?" e a resposta gerada não respondeu à pergunta de maneira relevante. A resposta abordou a pena aumentada para lesões corporais cometidas contra cônjuge, companheiro ou parente e citou lei e artigo específicos, o que é completamente desvinculado do tópico original d

In [25]:
df_input_veredict = pd.DataFrame(input_hallucination_veredict)

df_input_veredict

,question,answer,llm_response
0,O que é mucosidade doméstica?,Violência doméstica é qualquer ação ou omissão...,"Sim, houve alucinação de input. A resposta ger..."
1,O que é considerado crime a partir de uma asso...,Associação criminosa (organização voltada à pr...,A pergunta é sobre o que é considerado crime a...
2,O que caracteriza a lesão corporal em contexto...,Lesão corporal em violência doméstica tem pena...,"Sim, houve alucinação de input.\n\nA pergunta ..."
